# Задание
1. https://docs.opencv.org/3.4/dc/dff/tutorial_py_pyramids.html

2. https://docs.opencv.org/3.4/d3/db4/tutorial_py_watershed.html

3. https://docs.opencv.org/4.x/d8/d01/tutorial_discrete_fourier_transform.html

In [1]:
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt

# pt1

In [7]:
img = cv.imread('resources/lab6/messi5.png')
assert img is not None, "file could not be read, check with os.path.exists()"


lower_reso = cv.pyrDown(img)
higher_reso2 = cv.pyrUp(lower_reso)


# Для проверки результата вы можете показать изображение
cv.imshow('Lower Resolution', lower_reso)
cv.waitKey(0)
cv.destroyAllWindows()

In [8]:
A = cv.imread('resources/lab6/apple.png')
B = cv.imread('resources/lab6/orange.png')
assert A is not None, "file could not be read, check with os.path.exists()"
assert B is not None, "file could not be read, check with os.path.exists()"
# generate Gaussian pyramid for A
G = A.copy()
gpA = [G]
for i in range(6):
    G = cv.pyrDown(G)
    gpA.append(G)
# generate Gaussian pyramid for B
G = B.copy()
gpB = [G]
for i in range(6):
    G = cv.pyrDown(G)
    gpB.append(G)
# generate Laplacian Pyramid for A
lpA = [gpA[5]]
for i in range(5,0,-1):
    GE = cv.pyrUp(gpA[i])
    L = cv.subtract(gpA[i-1],GE)
    lpA.append(L)
# generate Laplacian Pyramid for B
lpB = [gpB[5]]
for i in range(5,0,-1):
    GE = cv.pyrUp(gpB[i])
    L = cv.subtract(gpB[i-1],GE)
    lpB.append(L)
# Now add left and right halves of images in each level
LS = []
for la,lb in zip(lpA,lpB):
    rows,cols,dpt = la.shape
    ls = np.hstack((la[:,0:cols//2], lb[:,cols//2:]))
    LS.append(ls)
# now reconstruct
ls_ = LS[0]
for i in range(1,6):
    ls_ = cv.pyrUp(ls_)
    ls_ = cv.add(ls_, LS[i])
# image with direct connecting each half
real = np.hstack((A[:,:cols//2],B[:,cols//2:]))
cv.imwrite('Pyramid_blending2.jpg',ls_)
cv.imwrite('Direct_blending.jpg',real)

error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\core\src\arithm.cpp:661: error: (-209:Sizes of input arguments do not match) The operation is neither 'array op array' (where arrays have the same size and the same number of channels), nor 'array op scalar', nor 'scalar op array' in function 'cv::arithm_op'


# pt2

In [17]:
img = cv.imread('resources/lab6/money.png')
assert img is not None, "file could not be read, check with os.path.exists()"
gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
ret, thresh = cv.threshold(gray,0,255,cv.THRESH_BINARY_INV+cv.THRESH_OTSU)
cv.imshow('img', thresh)
cv.waitKey(0)
cv.destroyAllWindows()

In [ ]:
# noise removal
kernel = np.ones((3,3),np.uint8)
opening = cv.morphologyEx(thresh,cv.MORPH_OPEN,kernel, iterations = 2)
# cv.imshow('img', opening)
# cv.waitKey(0)
# cv.destroyAllWindows()

# sure background area
sure_bg = cv.dilate(opening,kernel,iterations=3)
# cv.imshow('img', sure_bg)
# cv.waitKey(0)
# cv.destroyAllWindows()

# Finding sure foreground area
dist_transform = cv.distanceTransform(opening,cv.DIST_L2,5)
ret, sure_fg = cv.threshold(dist_transform,0.7*dist_transform.max(),255,0)

# Finding unknown region
sure_fg = np.uint8(sure_fg)
unknown = cv.subtract(sure_bg,sure_fg)



In [12]:
# Marker labelling
ret, markers = cv.connectedComponents(sure_fg)
# Add one to all labels so that sure background is not 0, but 1
markers = markers+1
# Now, mark the region of unknown with zero
markers[unknown==255] = 0

In [13]:
markers = cv.watershed(img,markers)
img[markers == -1] = [255,0,0]